(lgm-intro-notebook)=
# Curve di crescita latente 

In [1]:
source("_common.R")
suppressPackageStartupMessages({
    library("lavaan")
    library("semPlot")
    library("knitr")
    library("markdown")
    library("patchwork")
    library("psych")
    library("DT")
    library("kableExtra")
    library("lme4")
    library("JWileymisc")
    library("multilevelTools")
    library("data.table")
    library("insight")
})
set.seed(42)

Quando si studiano le differenze inter-individuali nel senso tradizionale, si effettuano confronti tra individui. Tuttavia, quando l’attenzione è rivolta al confronto dei cambiamenti intra-individuali piuttosto che alle differenze tra le medie dei punteggi di diversi individui, sono necessarie informazioni longitudinali per descrivere le traiettorie di cambiamento inter-individuale. Per descrivere le trasformazioni all’interno dell’individuo, è necessario un modello statistico in grado di tenere conto sia delle differenze tra individui sia delle trasformazioni inter-individuali. Per risolvere il problema descritto sopra, sono stati proposti due framework statistici: i modelli misti e un’estensione del framework SEM per dati longitudinali. Nell’ultima parte della dispensa verrà fornita un’introduzione a questi metodi statistici, con particolare attenzione al framework SEM per dati longitudinali.

## Misurare il cambiamento

Il modo più ovvio per misurare il cambiamento individuale tra due occasioni è quello di calcolare la differenza tra i punteggi ottenuti nelle due rilevazioni temporali. Tuttavia, questo approccio presenta un problema: non disponiamo del punteggio vero dell’individuo nei due momenti temporali, ma solo di un punteggio influenzato dall’errore di misurazione. L’errore di misura può ridurre drasticamente la determinazione delle differenze individuali nel cambiamento. 

Il problema di descrivere il cambiamento al di là delle distorsioni derivanti dall'errore di misura è stato affrontato introducendo una classe di modelli SEM per dati longitudinali chiamati *Latent Growth Models* (LGM). I modelli LCM assumeno che vi è una traiettoria individuale di cambiamento latente per ciascun individuo. Da ciò deriva che i dati in qualsiasi punto temporale sono assunti come la somma di (a) punteggi non osservati o latenti che rappresentano il livello dell’individuo in quel punto del tempo, (b) punteggi latenti che rappresentano il cambiamento individuale nel tempo e (c) caratteristiche uniche non osservate ma indipendenti delle misurazioni. 

Questa descrizione del cambiamento conduce alla formulazione di un modello SEM nel quale vengono ipotizzati tre tipi di fattori latenti: 

- un fattore latente comune che rappresenta al livello latente dell'individuo in un momento temporale (punto *a*),
- un fattore latente comune che rappresenta il cambiamento latente del livello dell'individuo tra $t-1$ e $t$ (punto *b*),
- un un fattore latente specifico che rappresenta l'errore di misura in ciascun punto $t$ (punto *c*).

Il punteggio osservato della persona $i$ nel momento $t$ è uguale a

$$
x_{it} = \tau_i + (1)\xi_1 + (t)\xi_2 + \delta_{it}
$$

dove

- $\xi_1$ corrisponde al fattore comune che descrive le intercette in funzione del tempo, con saturazioni fattoriali uguali a 1. I punteggi fattoriali su $\xi_1$ corrispondono dunque al punteggio vero di ciascun individuo indipendente dal tempo.
- $\xi_2$ corrisponde al fattore comune che descrive il cambiamento in funzione del tempo, con saturazioni fattoriali uguali a 0, 1, 2, ..., T. I punteggi fattoriali su $\xi_e$ corrispondono al cambiamento del punteggio vero di ciascun individuo in corrispondenza dell'aumento di un'unità nella metrica del tempo.
- $\delta_{it}$ rappresenta l'errore di misura dell'individuo $i$ nel tempo $t$.

Vedremo che, nel modello, le intercette osservate sono vincolate ad essere 0, per cui $\tau_i = 0$. Se ci sono cinque misurazioni temporali, per esempio, il modello diventa

$$
\begin{align}
x_{1} &= 0 + (1)\xi_{1} + (0)\xi_{2} + \delta_{1} \notag\\
x_{2} &= 0 + (1)\xi_{1} + (1)\xi_{2} + \delta_{2} \notag\\
x_{3} &= 0 + (1)\xi_{1} + (2)\xi_{2} + \delta_{3} \notag\\
x_{4} &= 0 + (1)\xi_{1} + (3)\xi_{2} + \delta_{4} \notag\\
x_{5} &= 0 + (1)\xi_{1} + (4)\xi_{2} + \delta_{5} \notag
\end{align}
$$

Inoltre, il modello ipotizza che $\xi_1$ e $\xi_e$ siano correlati, come indicato nella matrice di inter-correlazione fattoriale

$$
\mathbf{\Phi} =
\begin{bmatrix}
\phi_{11} & \\
\phi_{21} & \phi_{22}
\end{bmatrix}
$$

dove $\phi_{11}$ è la varianza dell’intercetta latente, $\phi_{22}$ è la varianza della pendenza latente e $\phi_{21}$ è la covarianza dell’intercetta e della pendenza.

La figura {ref}`lgm-fig` mostra il diagramma di percorso che rappresenta il modello LGM che stiamo discutendo:


```{figure} images/lgm_stat_mod.png
---
height: 400px
name: lgm-fig
---
Modello di crescita latente
```

Il modello di crescita latente definito dalle equazioni precedenti porta a una serie di previsioni sulla struttura delle medie e delle covarianze dei dati osservati. Queste previsioni del modello vengono utilizzate nel contesto della modellizzazione delle equazioni strutturali per calcolare le stime dei parametri e gli indici di adattamento del modello. La struttura delle covarianze ($\boldsymbol{\Sigma}$) prevista dal modello di crescita latente è

$$
\boldsymbol{\Sigma} = \boldsymbol{\Lambda} \boldsymbol{\Phi} \boldsymbol{\Lambda}' + \boldsymbol{\Theta}.
$$

Diversi tipi di modelli di crescita latente possono essere specificati cambiando le dimensioni e i valori contenuti nella matrice $\boldsymbol{Λ}$. Per il modello senza crescita, $\boldsymbol{Λ}$ è una matrice T × 1 (colonna) perché c’è solo una variabile latente nel modello senza crescita. Per il modello a crescita lineare, $\boldsymbol{Λ}$ è una matrice T × k, con k rilevazioni temporali, dove le saturazioni fattoriali sono fisse e uguali alla separazione temporale tra le rilevazioni. Per esempio, con cinque misurazioni temporali, le saturazioni fattoriali saranno specificate ai valori 0, 1, 2, 3, 4. Si noti che, in questo modo, si assume che tutte le unità di osservazione siano state misurate nella stessa occasione temporale. A differenza dei modelli CFA tradizionali in cui l’interpretazione si concentra sulle saturazioni fattoriali, nei modelli di crescita latente le saturazioni fattoriali sono fisse

$$
\Lambda =
\begin{pmatrix}
1 & 0 \\
1 & 1 \\
1 & 2 \\
1 & 3 \\
1 & 4 \\
\end{pmatrix}
$$

il che implica che l’attenzione è rivolta all’intercetta latente e al fattore di pendenza latente.


## Domande della ricerca

I modelli LGM possono dunque essere usati per per studiare il cambiamento nel tempo tramite l'analisi dei dati longitudinali in cui una misura di esito viene osservata in diversi momenti nel tempo. {cite:t}`grimm2016growth` elencano cinque motivi per usare i modelli LGM.

1. L’identificazione diretta del cambiamento e della stabilità intra-individuale. Misurando lo stesso individuo ripetutamente, i ricercatori possono identificare se e come specifici attributi dell’individuo cambiano o rimangono gli stessi nel tempo. Le teorie dello sviluppo spesso descrivono il cambiamento come un processo incrementale o trasformazionale. Il cambiamento incrementale viene identificato come un cambiamento nella grandezza di un costrutto lungo un continuum durante un intervallo di tempo specifico. Il cambiamento trasformazionale viene identificato come una transizione tra stati discreti durante un intervallo di tempo specifico (ad esempio, la teoria dello sviluppo a stadi di Piaget; Piaget, 1952). L’obiettivo principale è ottenere una descrizione accurata e parsimoniosa di come e quando gli attributi dell’individuo cambiano nel tempo. La stabilità e la costanza nel tempo sono casi speciali di cambiamento intra-individuale.
2. Una volta identificato il modello di cambiamento intra-individuale, una domanda logica successiva è se diversi individui cambiano in modi diversi. Pertanto, il secondo motivo per la ricerca longitudinale è l’identificazione delle differenze interindividuali nel cambiamento intra-individuale. Ci si chiede se i diversi individui cambino in quantità o direzioni diverse o se passino da uno stadio all’altro in momenti diversi.
3. Il terzo motivo per la ricerca longitudinale è l’analisi delle interrelazioni nel cambiamento comportamentale. Questo approccio si basa sull’idea che ci sono cambiamenti simultanei e/o sequenziali in più costrutti. L’analisi richiede l’analisi simultanea di più variabili e la valutazione di come i cambiamenti in una variabile influenzino i cambiamenti in un’altra variabile.
4. Il quarto motivo è l’analisi delle cause del cambiamento intra-individuale. L’obiettivo è identificare i fattori e/o i meccanismi variabili nel tempo che influenzano i cambiamenti intra-individuali. Si assume che i cambiamenti procedano a ritmi diversi in periodi di tempo diversi.
5. Il quinto motivo per la ricerca longitudinale è l’analisi delle cause delle differenze interindividuali nel cambiamento intra-individuale. Poiché gli individui differiscono nel modo in cui cambiano nel tempo, i ricercatori sono interessati a identificare i fattori che possono spiegare queste differenze. L’obiettivo è identificare le variabili invarianti nel tempo che sono correlate a specifici aspetti del cambiamento all’interno della persona. Ad esempio, le caratteristiche demografiche, gli interventi sperimentali e le caratteristiche dei contesti degli individui possono influenzare come e quando il cambiamento avviene. Le domande di ricerca derivanti dal quinto motivo sono spesso esaminate attraverso l’inclusione di covariate invarianti nel tempo.

